In [26]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [27]:
# 조회 기준 월평균 데이터 조회

from meteostat import Point, Daily
from datetime import datetime, timedelta
import pandas as pd
import requests

# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# 1. IP 기반으로 위도, 경도, 도시명 가져오기
def get_lat_lon_from_ip():
    """IP 기반으로 위도와 경도, 도시명 반환"""
    try:
        response = requests.get("https://ipinfo.io/json", timeout=5)
        data = response.json()
        lat, lon = map(float, data["loc"].split(","))
        city = data.get("city", "알 수 없음")
        print(f"📍 자동 위치 감지: {city} (위도 {lat}, 경도 {lon})")
        return city, lat, lon
    except Exception as e:
        print("❌ IP 기반 위치 조회 실패:", e)
        return None, None, None

# 2. 도시명을 위도/경도로 변환 (수동 입력 시)
def get_coords_from_city(city_name):
    try:
        url = f"https://nominatim.openstreetmap.org/search?q={city_name}&format=json&limit=1"
        headers = {"User-Agent": "weather-capstone-app"}
        response = requests.get(url, headers=headers).json()

        if response:
            lat = float(response[0]["lat"])
            lon = float(response[0]["lon"])
            return city_name, lat, lon
        else:
            raise ValueError("도시 이름을 찾을 수 없습니다.")
    except Exception as e:
        print("❌ 도시명 변환 실패:", e)
        new_city = input("🚨 유효한 도시명을 다시 입력해 주세요: ")
        return get_coords_from_city(new_city)

# 3. 사용자 위치 자동 감지 또는 수동 입력
def get_location():
    city, lat, lon = get_lat_lon_from_ip()
    if city is None or lat is None or lon is None:
        user_city = input("🌍 자동 위치 조회 실패! 직접 도시명을 입력하세요: ")
        city, lat, lon = get_coords_from_city(user_city)
    return city, lat, lon

today = datetime.today()
start = today - timedelta(days=30)
end = today

# 5. 위치 조회 및 출력
city, lat, lon = get_location()
print(f"📍 사용된 위치: {city} (위도 {lat}, 경도 {lon})")
print(f"📅 조회 기간: {start.date()} ~ {end.date()}")

# 6. Meteostat 날씨 데이터 수집
location = Point(lat, lon)
data = Daily(location, start, end).fetch()
data.index = pd.to_datetime(data.index)

# 7. 평균값 계산 및 출력
mean_values = data.mean(numeric_only=True)
print("\n📊 31일 평균 기상 데이터:")
print(mean_values.round(2))

📍 자동 위치 감지: Cheonan (위도 36.8065, 경도 127.1522)
📍 사용된 위치: Cheonan (위도 36.8065, 경도 127.1522)
📅 조회 기간: 2025-04-21 ~ 2025-05-21

📊 31일 평균 기상 데이터:
tavg      16.31
tmin      10.90
tmax      21.81
prcp       2.60
snow        NaN
wdir     205.87
wspd       9.97
wpgt        NaN
pres    1010.51
tsun        NaN
dtype: float64


In [28]:
# 1. FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated...
# 날짜 형식의 자동 파싱 기능이 곧 제거될 예정
# pandas 측에서 사전 경고

# 2. Warning: Cannot load daily/RKSK0.csv.gz from https://bulk.meteostat.net/v2/
# meteostat의 특정 관측소에서 압축된 일일 데이터가 부분적인 손상이나 다운로드 불가능한 오류

In [29]:
import json
import os
import math
from datetime import datetime

# 평균 기상 데이터 (pandas Series → dict)
avg_weather_dict = mean_values.round(2).to_dict()

# NaN → None 으로 변환
def sanitize_nan(data):
    return {
        k: (None if isinstance(v, float) and math.isnan(v) else v)
        for k, v in data.items()
    }

avg_weather_dict_cleaned = sanitize_nan(avg_weather_dict)

# 저장할 파일 경로
date_str = today.strftime("%Y%m%d")
filename = f"monthly_avg_weather_{city}_{date_str}.json"

# JSON으로 저장
try:
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(avg_weather_dict_cleaned, f, ensure_ascii=False, indent=4)

    print(f"✅ 파일이 성공적으로 저장되었습니다: {filename}")

except Exception as e:
    print(f"❌ 오류 발생: {e}")


✅ 파일이 성공적으로 저장되었습니다: monthly_avg_weather_Cheonan_20250521.json


In [30]:
print(json.dumps(avg_weather_dict, indent=4))

{
    "tavg": 16.31,
    "tmin": 10.9,
    "tmax": 21.81,
    "prcp": 2.6,
    "snow": NaN,
    "wdir": 205.87,
    "wspd": 9.97,
    "wpgt": NaN,
    "pres": 1010.51,
    "tsun": NaN
}
